## Ataques de tiburones:
Vamos a determinar el número de ataques distribuidos por decada a partir del año 1900 y contabilizando el tipo de ataques
y si estos han sido fatales.
Ver que meses son los más peligrosos y sexo etc


In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

Debemos añadir la codificación para obtener los datos correctamente del csv (ISO-8859-1)

In [2]:
data = pd.read_csv('./data/GSAF5.csv', encoding='ISO-8859-1')
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


# Data Cleaning
Consultamos la información del dataframe, para ver los nombre de las columnas y sus tipo de datos

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

Ahora buscamos los valores nulos para decidir que columnas eliminar

In [44]:
def nulos(dataframe):
    null_cols = dataframe.isnull().sum()
    print(null_cols[null_cols > 0])
    
nulos(data)

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64


In [5]:
#Eliminio las columnas que tienen más de 3000 registros nulos  

drop_cols = list(null_cols[null_cols > 3000].index)
nueva_data = data.drop(drop_cols, axis=1)

In [6]:
#También eliminio otras columnas que no voy a utilizar
nueva_data = nueva_data.drop(['Species ', 'Injury', 'Age', 'Activity', 'Name', 'Date', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order'], axis=1)
display(nueva_data.head())
print(nueva_data.shape)

,Case Number,Year,Type,Country,Area,Location,Sex,Fatal (Y/N)
0,2016.09.18.c,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",M,N
1,2016.09.18.b,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",M,N
2,2016.09.18.a,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",M,N
3,2016.09.17,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,M,N
4,2016.09.15,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,M,N


(5992, 8)


In [28]:
#Compruebo si existen registros duplicados
def remover_duplicados(dataframe, columnas = []):
    antes = len(dataframe)
    dataframe = dataframe[columnas].drop_duplicates()
    despues = len(dataframe)
    total = antes - despues
    print('Registros duplicados eliminados: {}'.format(total))

remover_duplicados(data)

Registros duplicados eliminados: 0


In [8]:
test = nueva_data[(nueva_data['Year'] == 0)] #valores incorrectos
print(len(test))

124


In [9]:
#Low variance y Valores extremos
low_variance = []

for col in nueva_data._get_numeric_data():
    minimum = min(nueva_data[col])
    ninety_perc = np.percentile(nueva_data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)

print('Low Variance', low_variance)

stats = nueva_data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

Low Variance []


,count,mean,std,min,25%,50%,75%,max,IQR
Year,5992.0,1925.204606,286.473712,0.0,1942.0,1975.0,2003.0,2016.0,61.0


In [11]:
data_country=nueva_data.groupby('Country').count()
#print(data_country)

outliers = pd.DataFrame(columns=nueva_data.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = nueva_data[(nueva_data[col] < lower) | 
                   (nueva_data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

print(len(outliers))

238


In [12]:
#Columnas restantes
print(nueva_data.columns)
print(len(nueva_data.columns))

Index(['Case Number', 'Year', 'Type', 'Country', 'Area', 'Location', 'Sex ',
       'Fatal (Y/N)'],
      dtype='object')
8


In [13]:
#Renombro varias columnas, unas porque tienen un espacio al final y otras porque contienen otros caracteres
nueva_data = nueva_data.rename(columns={'Fatal (Y/N)':'Fatal', 
                                  'Sex ':'Sex'})
print(nueva_data.columns)
print(nueva_data.shape)

Index(['Case Number', 'Year', 'Type', 'Country', 'Area', 'Location', 'Sex',
       'Fatal'],
      dtype='object')
(5992, 8)


In [39]:
#Compruebo los diferentes tipos de ataque y los difrentes valores para la columan Fatal
tipos_ataque = nueva_data.Type.unique()
print(tipos_ataque)
set(nueva_data['Type'])

nueva_data['Fatal'].value_counts()


['Unprovoked' 'Boat' 'Provoked' 'Invalid' 'Sea Disaster' 'Boating']


N          4325
Y          1552
UNKNOWN      96
Name: Fatal, dtype: int64

In [36]:
print(set(nueva_data['Sex']))

s = nueva_data[(nueva_data['Sex'].isnull() == True)]

print(len(s))


{nan, 'N', '.', 'F', 'M', 'lli', 'M '}
567


In [42]:
#Modificamos los datos en la columna Fatal para unificarlos (N, Y, UNKNOWN)
nueva_data['Fatal'] = nueva_data['Fatal'].str.strip().str.upper()
nueva_data['Fatal'] = nueva_data['Fatal'].str.replace('#VALUE!', 'UNKNOWN')
nueva_data['Fatal'] = nueva_data['Fatal'].str.replace('F', 'UNKNOWN')

#Los valores nulos también los pasamos a UNKNOWN
nueva_data['Fatal'] = nueva_data['Fatal'].fillna('UNKNOWN')
print(nueva_data['Fatal'].value_counts())

N          4325
Y          1552
UNKNOWN     115
Name: Fatal, dtype: int64


In [45]:
nulos(nueva_data)

Country      43
Area        402
Location    496
Sex         567
dtype: int64


In [29]:
#Volvemos a comprobar si hay duplicados después del cambio en la columna Fatal
remover_duplicados(nueva_data, nueva_data.columns)

Registros duplicados eliminados: 0


# Manipulation

In [ ]:
#También filtro los datos a partir de un año en concreto ya que anteriormente
#a ese año los datos no son significativos
#Y los ordeno por año ascendentemente
datos = nueva_data[(nueva_data.Year > 1889) & (nueva_data.Year <= 2016) & (nueva_data.Type != 'Invalid')]
datos = datos.sort_values(by=['Year'])
datos.head()
print(len(datos))

In [ ]:
remover_duplicados(datos)

<img src="https://thumbs-prod.si-cdn.com/Qvj8K8U9cFWVzIHi8CrApJp1wRQ=/800x600/filters:no_upscale():focal(1542x1697:1543x1698)/https://public-media.si-cdn.com/filer/d0/06/d00620a0-8dc9-4be8-a3c1-cd075845b348/42-53008611.jpg" width="1%" />